In [1]:
import sys
from chart_studio import plotly
import plotly.graph_objs as go
import plotly.offline as pltlyoff
import numpy as np
import matplotlib.pyplot as plt

### Custom files ###
dir_GPPlotly = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_plotly\\'
sys.path.append(dir_GPPlotly)

dir_gp_constr = 'C:\\Users\\tln229\\Downloads\\Python Tests\\Constrained GP_2\\GP_2\\gp_constr\\'
sys.path.append(dir_gp_constr)

from GPPlotly.plottingfunctions import PlotGP2d, add_traces_to_fig
from GPConstr.model import GPmodel, Constraint
from GPConstr.kern import kernel_RBF

Loading constrained GP module from C:\Users\tln229\Downloads\Python Tests\Constrained GP_2\GP_2\gp_constr


Unable to determine R home: [WinError 2] The system cannot find the file specified


Loading R wrapper...


R[write to console]: Loading required package: mvtnorm

R[write to console]: Loading required package: Matrix

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: gmm

R[write to console]: Loading required package: sandwich

R[write to console]: 
Attaching package: 'TruncatedNormal'


R[write to console]: The following objects are masked from 'package:tmvtnorm':

    dtmvnorm, dtmvt, ptmvnorm, ptmvt, rtmvnorm, rtmvt


R[write to console]: The following objects are masked from 'package:mvtnorm':

    pmvnorm, pmvt




Running R from rpy2: R version 4.2.2 (2022-10-31 ucrt)


In [2]:
# ---------- DATA ----------
def fun(x):
    return (np.arctan(20*x - 10) - np.arctan(-10))/3

x_train = np.array([1/(n+2) for n in range(7)] ) + 0.1
y_train = fun(x_train)

# For plotting
x_test = np.linspace(0, 1, 500)
y_true = fun(x_test)

In [3]:
# ---------- MODEL ----------
ker = kernel_RBF(variance = 0.5, lengthscale = [0.1])
model = GPmodel(kernel = ker, likelihood = 1, mean = 0)

model.X_training = x_train.reshape(-1, 1)
model.Y_training = y_train

In [4]:
# ---------- MODEL OPTIMIZE ------------
model.optimize(include_constraint = False, fix_likelihood = False)
print(model)

..Running optimization for unconstrained GP ... DONE - Total time: 0.025 seconds
----- GP model ----- 
 mean = 0 
 likelihood = 1e-06 
 kernel: 
   type = RBF 
   input dim = 1 
   lenghtscale = [0.2553908] 
   variance = 0.8630355457207904 
 constraint: 
   No constraints 
---------------------


In [5]:
# ---------- CONSTRAINTS ----------
def constant_function(val):
    """ Return the constant function"""
    def fun(x):
        return np.array([val]*x.shape[0])
    return fun

model.constr_bounded    = Constraint(LB = constant_function(0), UB = constant_function(1))
model.constr_bounded.Xv = np.linspace(0, 1, 20).reshape(-1, 1)

model.constr_deriv       = [Constraint(LB = constant_function(0), UB = constant_function(float('Inf')))]
model.constr_deriv[0].Xv = np.linspace(0, 1, 15).reshape(-1, 1)

model.constr_likelihood = 1E-6

In [6]:
mean, var, perc, mode, samples, times = model.calc_posterior_constrained(x_test.reshape(-1, 1), compute_mode = False, num_samples = 1000, save_samples = 5, algorithm = 'minimax_tilting', resample = False)

mean = np.array(mean).flatten()
p_lower = perc[0]
median = perc[1]
p_upper = perc[2]
p_label = '[p{}, p{}]'.format(10, 90)

samplepaths_Z = np.array(samples)

fig_both = PlotGP2d(x_mean = x_test,
                    mean = mean,
                    x_obs = model.X_training,
                    y_obs = model.Y_training, 
                    p_lower = p_lower,
                    p_upper = p_upper,
                    p_label = p_label,
                    samplepaths = samplepaths_Z,
                    x_true = x_test,
                    y_true = y_true,
                    title = 'Both constraints',
                    xrange = [0, 1],
                    yrange = [-0.5, 1.5],
                    smoothing = True)

trace_UB = go.Scatter(x = x_test, y = model.constr_bounded.UB(x_test), name = 'Upper bound')
trace_LB = go.Scatter(x = x_test, y = model.constr_bounded.LB(x_test), name = 'Lower bound')
trace_XV_bounded = go.Scatter(x = model.constr_bounded.Xv.flatten(), y = np.zeros(model.constr_bounded.Xv.shape[0]), mode = 'markers', name = 'Xv - boundedness')
trace_XV_mon = go.Scatter(x = model.constr_deriv[0].Xv.flatten(), y = np.zeros(model.constr_deriv[0].Xv.shape[0]), mode = 'markers', name = 'Xv - monotonicity')

fig_both = add_traces_to_fig(fig_both, [trace_UB, trace_LB, trace_XV_bounded, trace_XV_mon])

pltlyoff.iplot(fig_both, filename='')

..Running calculation of K_w ... SKIP - (cached)
..Running calculation of Cholesky factor for K_w ... SKIP - (cached)
..Running preparation step 1 - dependence on (XS, X) ... DONE - time: 0.007 seconds
..Running preparation step 2 - dependence on (XV, X) ... DONE - time: 0.001 seconds
..Running preparation step 3 - dependence on (XS, XV, X) ... DONE - time: 0.002 seconds
..sampling 1000 times from truncated constraint distribution C~|C, Y DONE - time: 1 minutes 40.8 seconds
..sampling 1000 times from constrained GP f*|C, Y DONE - time: 0.460 seconds
..computing statistics from samples DONE - time: 0.012 seconds
 DONE - Total time: 1 minutes 41.3 seconds
